# FUNCIONES

In [1]:
import os
import pprint
import numpy as np
import copy
from math import sqrt
from scipy.linalg import solve_triangular

### Creación matrices

In [2]:
def crea_matriz(renglones,columnas,maximo_valor,minimo_valor,entero=False):
    """
    Función de apoyo para genear matrices aleatorias
                            
    params: renglones       no. de renglones de la matriz
            columnas        no. de renglones de la matriz
            maximo_valor    valor máximo de las entradas de la matriz
            minimo_valor    valor mínimo de las entradas de la matriz
            entero          Indica si las entradas serán enteras (True) o no
            
    return: M               Matriz con numeros al azar
    """
    M=np.zeros((renglones, columnas))
    for i in range(renglones):
        for j in range(columnas):
            if entero:
                M[i][j]=(np.random.rand(1)*(maximo_valor+1-minimo_valor)+minimo_valor)//1
            else:
                M[i][j]=np.random.rand(1)*(maximo_valor-minimo_valor)+minimo_valor
    return M

### Factorización QR

In [3]:
def house(x):
    """
    Función que calcula la proyección de householder
    
    params: x       vector al que se le hará la reflexión householder
                    
    return: Beta    constante utilizada para obtener v
            v       vector que representa la reflexión de householder
    """
    m=len(x)
    norm_2_m=x[1:m].dot(np.transpose(x[1:m]))
    v=np.concatenate((1,x[1:m]), axis=None)
    Beta=0
    if (norm_2_m==0 and x[0]>=0):
        Beta=0
    elif (norm_2_m==0 and x[0]<0):
        Beta=2
    else:
        norm_x=np.sqrt(pow(x[0],2)+norm_2_m)
        if (x[0]<=0):
            v[0]=x[0]-norm_x
        else:
            v[0]=-norm_2_m/(x[0]+norm_x)
        Beta=2*pow(v[0],2)/(norm_2_m+pow(v[0],2))
        v=v/v[0]
    return Beta, v

In [4]:
def factorizacion_QR(A):
    """
    Función que genera una matriz que contendrá información escencial de las proyecciones householder
    (vectores v's) y componentes de la matriz triangular superior R, del estilo:
    [r11      r12      r13      r14    ]
    [v_2_(1)  r22      r23      r24    ]
    [v_3_(1)  v_3_(2)  r33      r34    ]
    [v_4_(1)  v_4_(2)  v_4_(3)  r44    ]
    [v_5_(1)  v_5_(2)  v_5_(3)  v_5_(4)]
    
    params: A      Matriz (mxn) de la que se desea obtener factorización QR
            
    return: A_r_v  Matriz (mxn) con la información esencial (es igual a la matriz R, pero en lugar de tener ceros
                   en la parte inferior, contiene info de los vectores householder que serán útiles para
                   futuros cálculos, que entre otros están el calcular la matriz ortonormal Q)
    """
    m=A.shape[0]
    n=A.shape[1]
    A_r_v=copy.copy(A)
    for j in range(n):
        beta, v=house(A_r_v[j:m,j])
        A_r_v[j:m,j:n]=A_r_v[j:m,j:n]-beta*(np.outer(v,v)@A_r_v[j:m,j:n])
        A_r_v[(j+1):m,j]=v[1:(m-j)]
    return A_r_v

In [5]:
def QT_C(A_r_v,C):
    """
    Función que calcula el producto matricial de Q_transpuesta por una matriz dada C
                            
    params: A_r_v   Matriz (mxn) con la información esencial (es igual a la matriz R, pero en lugar de tener ceros
                    en la parte inferior, contiene información de los vectores householder que serán útiles para
                    futuros cálculos, que entre otros están el calcular la matriz ortonormal Q)
            C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la función devolverá Q)

    return: M       Matriz con número al azar
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    QT_por_C=np.eye(m)
    for j in range(n-1,-1,-1):
        v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
        beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
        QT_por_C[j:m,j:m]=C[j:m,j:m]-beta*np.outer(v,v)@C[j:m,j:m]
    return QT_por_C

In [6]:
def Q_j(A_r_v,j):
    """
    Función que calcula la matriz Qj (en el proceso de obtención de factorización QR se van obteniendo n Qj's,
    que si se multiplican todas da por resultado Q=Q1*Q2*...*Qn)
                            
    params: A_r_v   Matriz (mxn) con la información esencial (es igual a la matriz R, pero en lugar de tener ceros
                    en la parte inferior, contiene información de los vectores householder que serán útiles para
                    futuros cálculos, que entre otros están el calcular la matriz ortonormal Q)
            C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la función devolverá Q)

    return: Qj      Matriz Q de la j-esima iteración del proceso iterativo de factorización QR
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    Qj=np.eye(m)
    v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
    beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
    Qj[j:m,j:m]=np.eye(m-j)-beta*np.outer(v,v)
    return Qj

### Funciones para solución de Sistemas de Ecuaciones Lineales

In [7]:
def Solucion_SEL_QR_nxn(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineales (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientes de las ecuaciones
            b   vector (nx1) constantes del sistema

    return: x   vector que satisface (Ax=b)
    """
    A_r_v=factorizacion_QR(A)
    m=A_r_v.shape[0]
    #Q=np.transpose(QT_C(A_r_v,np.eye(m)))
    #R=np.transpose(Q)@A
    n=A_r_v.shape[0]
    Q=np.eye(m)
    R=copy.copy(A)
    for j in range(m):
        Qj=Q_j(A_r_v,j)
        Q=Q@Qj
        R=Q_j(A_r_v,j)@R
    b_prima=np.transpose(Q)@b
    x = solve_triangular(R, np.transpose(Q)@b)
    return x

#### Eliminación por bloques

In [8]:
def bloques(A, b=False, n1=False, n2=False):
    """
    Esta es la función para la creación de bloques usando un arreglo de numpy
    
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
            n1  Numero de renglones que tendrá el 1er bloque
            n2  Numero de renglones que tendrá el 2do bloque
    
    return: A11 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            b1  Fraccion del vector dividido
            b2  Fraccion del vector dividido
    """

    # Primero definimos el n
    m,n = A.shape

    # Condiciones de A
    # Si no se dan los n deseados, se intentan hacer los bloques casi iguales
    if  not (n1&n2):
        n1 = n//2
        n2 = n - n1
    # Los bloques deben cumplir la condicion de tamaño
    elif n1+n1 != n:
        sys.exit('n1 + n2 debe ser igual a n')
    else:
        None

    # Condiciones de b
    if  b is False:
        b1 = None
        b2 = None
        print('condicion1')
    elif len(b) == m:
        b1 = b[:n1]
        b2 = b[n1:m]
    else:
        sys.exit('los renglones de A y b deben ser del mismo tamaño')

    A11 = A[:n1,:n1]
    A12 = A[:n1,n1:n]
    A21 = A[n1:m,:n1]
    A22 = A[n1:m,n1:n]

    return A11,A12,A21,A22,b1,b2

In [9]:
def eliminacion_bloques(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
    
    return: x1 Solucion al 1er sistema de ecuaciones obtenido con la división por bloques
            x2 Solucion al 2do sistema de ecuaciones obtenido con la división por bloques
    """
    if np.linalg.det(A)==0:
        sys.exit('A debe ser no singular')

    A11,A12,A21,A22,b1,b2 = bloques(A,b)

    if np.linalg.det(A11)==0:
        ys.exit('A11 debe ser no singular')

    ## 1. Calcular A11^{-1}A12 y A11^{-1}b1 teniendo cuidado en no calcular la inversa sino un sistema de ecuaciones lineales
    ## Aquí se debe usar el método QR una vez que esté desarrollado

    ## Definimos y = A11^{-1}b1, por tanto A11y=b1. Resolviendo el sistema anterior para 11y:
    y = Solucion_SEL_QR_nxn(A11,b1)
    #y = np.linalg.solve(A11,b1)

    ## Definimos Y = A11^{-1}A12
    Y = Solucion_SEL_QR_nxn(A11,A12)
    #Y = np.linalg.solve(A11,A12)

    ## 2. Calcular el complemento de Schur del bloque A11 en A. Calcular b_hat
    S = A22 - A21@Y
    b_h = b2 - A21@y

    ## 3. Resolver Sx2 = b_hat
    x2 = Solucion_SEL_QR_nxn(S,b_h)
    #x2 = np.linalg.solve(S,b_h)

    ## 4. Resolver A11x1 = b1-A12X2
    x1 = Solucion_SEL_QR_nxn(A11,b1-A12@x2)
    #x1 = np.linalg.solve(A11,b1-A12@x2)

    return np.concatenate((x1,x2), axis=0)

# Prueba Precisión - Factorización QR

Verificaremos que la factorización QR de una matriz obtenida con la función de numpy, es igual a la obtenida con la función implementada por los programadores.

Empezaremos por generar una matriz aleatoria con la función crea_matriz

In [10]:
# Generamos una matriz aleatoria de tamaño definido (renglones y columnas) y valores aleatorios comprendidos en un rango, pudiendo ser sus entradas de tipo enteros o dobles
m=5
n=3
A=np.round(crea_matriz(m,n,6,-6,False),2)
A

array([[ 2.61, -1.09, -0.86],
       [-5.45,  1.42,  2.8 ],
       [-1.72,  0.2 , -2.33],
       [ 5.98, -1.13,  5.22],
       [ 4.05,  3.11, -1.45]])

### Implementación programadores

Utilazaremos la función factorizacion_QR para obtener la factorización de la matriz A creada anteriormente.

In [11]:
A_r_v = factorizacion_QR(A)
np.round(A_r_v,4)

array([[ 9.5727, -0.5317,  1.2375],
       [ 0.7827,  3.7297, -1.4217],
       [ 0.247 , -0.1804,  6.3093],
       [-0.8589,  0.8595,  0.2295],
       [-0.5817, -1.4873, -0.8567]])

Utilizamos la función QT_C para obetener Q.

In [12]:
Q=np.transpose(QT_C(A_r_v,np.eye(m)))
np.round(Q,4)

array([[ 0.2727, -0.5693, -0.1797,  0.6247,  0.4231],
       [-0.5693,  0.5544, -0.1406,  0.489 ,  0.3312],
       [-0.1797, -0.1406,  0.9556,  0.1543,  0.1045],
       [ 0.6247,  0.489 ,  0.1543,  0.4635, -0.3634],
       [ 0.4231,  0.3312,  0.1045, -0.3634,  0.7539]])

Ahora resolvemos y obtenemos R.

In [13]:
R=np.transpose(Q)@A
np.round(R,4)

array([[ 9.5727, -0.5317,  1.2375],
       [ 0.    ,  1.857 ,  4.4418],
       [-0.    ,  0.3379, -1.8119],
       [ 0.    , -1.6095,  3.4185],
       [ 0.    ,  2.7853, -2.6701]])

Calcularemos la multiplicación de las matrices Q y R para validar que obtenemos A.

In [14]:
print('Q@R:')
pprint.pprint(Q@R)
print('A:')
pprint.pprint(A)

Q@R:
array([[ 2.61, -1.09, -0.86],
       [-5.45,  1.42,  2.8 ],
       [-1.72,  0.2 , -2.33],
       [ 5.98, -1.13,  5.22],
       [ 4.05,  3.11, -1.45]])
A:
array([[ 2.61, -1.09, -0.86],
       [-5.45,  1.42,  2.8 ],
       [-1.72,  0.2 , -2.33],
       [ 5.98, -1.13,  5.22],
       [ 4.05,  3.11, -1.45]])


### Implementación Numpy

Obtenemos Q y R con la función *numpy.linalg.qr()*

In [15]:
Q_np, R_np = np.linalg.qr(A)
print('Q_np:')
pprint.pprint(Q_np)
print('R_np:')
pprint.pprint(R_np)

Q_np:
array([[-0.27265137,  0.25337866, -0.24688273],
       [ 0.56932949, -0.29956334,  0.62296543],
       [ 0.1796783 , -0.02800876, -0.32774543],
       [-0.62469547,  0.21391659,  0.65662448],
       [-0.42307971, -0.89415673, -0.11131136]])
R_np:
array([[-9.57266421,  0.53170151, -1.23749248],
       [ 0.        , -3.72971762,  1.42174929],
       [ 0.        ,  0.        ,  6.30925046]])


Calcularemos la multiplicación de las matrices Q y R para validar que obtenemos A.

In [16]:
print('Q_np@R_np:')
pprint.pprint(Q_np@R_np)
print('A:')
pprint.pprint(A)

Q_np@R_np:
array([[ 2.61, -1.09, -0.86],
       [-5.45,  1.42,  2.8 ],
       [-1.72,  0.2 , -2.33],
       [ 5.98, -1.13,  5.22],
       [ 4.05,  3.11, -1.45]])
A:
array([[ 2.61, -1.09, -0.86],
       [-5.45,  1.42,  2.8 ],
       [-1.72,  0.2 , -2.33],
       [ 5.98, -1.13,  5.22],
       [ 4.05,  3.11, -1.45]])


### Comparación

**Validación A = QR**

Valores obtenidos con la función implementada por los programadores.

In [17]:
print('Q@R:')
pprint.pprint(Q@R)
print('A:')
pprint.pprint(A)

Q@R:
array([[ 2.61, -1.09, -0.86],
       [-5.45,  1.42,  2.8 ],
       [-1.72,  0.2 , -2.33],
       [ 5.98, -1.13,  5.22],
       [ 4.05,  3.11, -1.45]])
A:
array([[ 2.61, -1.09, -0.86],
       [-5.45,  1.42,  2.8 ],
       [-1.72,  0.2 , -2.33],
       [ 5.98, -1.13,  5.22],
       [ 4.05,  3.11, -1.45]])


Valores obtenidos con la función de numpy.

In [18]:
print('Q_np@R_np:')
pprint.pprint(Q_np@R_np)
print('A:')
pprint.pprint(A)

Q_np@R_np:
array([[ 2.61, -1.09, -0.86],
       [-5.45,  1.42,  2.8 ],
       [-1.72,  0.2 , -2.33],
       [ 5.98, -1.13,  5.22],
       [ 4.05,  3.11, -1.45]])
A:
array([[ 2.61, -1.09, -0.86],
       [-5.45,  1.42,  2.8 ],
       [-1.72,  0.2 , -2.33],
       [ 5.98, -1.13,  5.22],
       [ 4.05,  3.11, -1.45]])


Las dos funciones cumplen que A = QR.

Podemos ver que obtenemos los mismos resultados al multiplicar las matrices Q y R obtenidas por cada función respectivamente.

**Validación Q**

Valores obtenidos con la función implementada por los programadores.

In [19]:
print('Q:')
pprint.pprint(Q)

Q:
array([[ 0.27265137, -0.56932949, -0.1796783 ,  0.62469547,  0.42307971],
       [-0.56932949,  0.55435942, -0.14064253,  0.4889781 ,  0.3311641 ],
       [-0.1796783 , -0.14064253,  0.95561373,  0.15431969,  0.10451417],
       [ 0.62469547,  0.4889781 ,  0.15431969,  0.4634699 , -0.36336905],
       [ 0.42307971,  0.3311641 ,  0.10451417, -0.36336905,  0.75390558]])


Valores obtenidos con la función de numpy.

In [20]:
print('Q_np:')
pprint.pprint(Q_np)

Q_np:
array([[-0.27265137,  0.25337866, -0.24688273],
       [ 0.56932949, -0.29956334,  0.62296543],
       [ 0.1796783 , -0.02800876, -0.32774543],
       [-0.62469547,  0.21391659,  0.65662448],
       [-0.42307971, -0.89415673, -0.11131136]])


Por alguna razón, al comparar los valores de Q obtenidos por las dos funciones tenemos diferencias. 


**Validación R**

Valores obtenidos con la función implementada por los programadores.

In [21]:
print('R:')
pprint.pprint(R)

R:
array([[ 9.57266421e+00, -5.31701509e-01,  1.23749248e+00],
       [ 6.35062829e-16,  1.85700611e+00,  4.44180459e+00],
       [-4.63210225e-17,  3.37917524e-01, -1.81185250e+00],
       [ 1.14423920e-15, -1.60950395e+00,  3.41853368e+00],
       [ 9.09689110e-16,  2.78525234e+00, -2.67005662e+00]])


Valores obtenidos con la función de numpy.

In [22]:
print('R_np:')
pprint.pprint(R_np)

R_np:
array([[-9.57266421,  0.53170151, -1.23749248],
       [ 0.        , -3.72971762,  1.42174929],
       [ 0.        ,  0.        ,  6.30925046]])


Por alguna razón, al comparar los valores de R obtenidos por las dos funciones tenemos diferencias. 


Tanto las funciones implementadas por los programadores para obtener la factorización $QR$ de una matriz, como la función *linalg.qr* de la librería numpy consideran una matriz de entrada A de dimensiones $mxn$. Como resultado se obtienen las siguientes dos matrices:

+ Función implementa por los programadores:
    + matriz ortogonal $Q$ de dimensiones $mxm$
    + matriz triangular superior $R$ de dimensiones $mxn$

+ Función linalg.qr de numpy
    + matriz con columnas ortonormales $Q$ de dimensiones $mxn$
    + matriz traingular superior $R$ con dimensiones $nxn$
    
 
En el caso de la función implementada en numpy, se obtiene lo que se conoce como "Factorización thin QR", en donde $Q$ ya no es una matriz ortogonal, si no una matriz con entradas ortonormales.

Lo anterior muestra por qué en las secciones de **Validación Q** y **Validación R** se obtienen matrices distintas; sin embargo, como se demuestra en la sección **Comparación A=QR**, al multiplicar uno u otro par de matrices $QR$, se recupera la matriz original $A$.